# Gia Gillis 

## Loan Interest Rate Analysis Part 1 of 3

### Data cleaning.

### For data cleaning the file, unneeded columns are removed and nulls are filled and/or removed

#### Import needed libraries.

In [48]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

#### Read csv file and force Loan Id (X2), Borrower Id (X3), and Reason (X16) columns to strings, and rename columns.

In [49]:
loans=pd.read_csv(r'data\loan_interest_rates.csv', 
                  dtype={'X2': str, 'X3':str, 'X16': str}, parse_dates=True)
loans.columns=['Interest Rate', 'Loan Id', 'Borrower Id', 'Requested', 'Funded', 'Investor Funded', 'Number of Payments',
              'Loan Grade', 'Loan Subgrade', 'Job', 'Years Employed', 'Home', 'Annual Income', 'Income Verified', 'Loan Date',
              'Reason', 'Loan Cat', 'Loan Title', 'State', 'Ratio', 'Late Payments', 'Credit Line Date', 'Months Del',
              'Months PR', 'Derog Recs', 'Credit Lines', 'Status']

In [50]:
loans.head()

,Interest Rate,Loan Id,Borrower Id,Requested,Funded,Investor Funded,Number of Payments,Loan Grade,Loan Subgrade,Job,...,Loan Title,State,Ratio,Late Payments,Credit Line Date,Months Del,Months PR,Derog Recs,Credit Lines,Status
0,11.89%,54734,80364,"$25,000","$25,000","$19,080",36 months,B,B4,NaN,...,Debt consolidation for on-time payer,CA,19.48,0.0,Feb-94,NaN,NaN,0.0,42.0,f
1,10.71%,55742,114426,"$7,000","$7,000",$673,36 months,B,B5,CNN,...,Credit Card payoff,NY,14.29,0.0,Oct-00,NaN,NaN,0.0,7.0,f
2,16.99%,57167,137225,"$25,000","$25,000","$24,725",36 months,D,D3,Web Programmer,...,mlue,NY,10.50,0.0,Jun-00,41.0,NaN,0.0,17.0,f
3,13.11%,57245,138150,"$1,200","$1,200","$1,200",36 months,C,C2,city of beaumont texas,...,zxcvb,TX,5.47,0.0,Jan-85,64.0,NaN,0.0,31.0,f
4,13.57%,57416,139635,"$10,800","$10,800","$10,692",36 months,C,C3,State Farm Insurance,...,Nicolechr1978,CT,11.63,0.0,Dec-96,58.0,NaN,0.0,40.0,f


In [51]:
loans.dtypes

Interest Rate          object
Loan Id                object
Borrower Id            object
Requested              object
Funded                 object
Investor Funded        object
Number of Payments     object
Loan Grade             object
Loan Subgrade          object
Job                    object
Years Employed         object
Home                   object
Annual Income         float64
Income Verified        object
Loan Date              object
Reason                 object
Loan Cat               object
Loan Title             object
State                  object
Ratio                 float64
Late Payments         float64
Credit Line Date       object
Months Del            float64
Months PR             float64
Derog Recs            float64
Credit Lines          float64
Status                 object
dtype: object

#### Find the null values in columns and count.

In [52]:
missing_data = loans.isnull()
null_columns=loans.columns[missing_data.any()]
loans[null_columns].isnull().sum()

Interest Rate          61010
Loan Id                    1
Borrower Id                1
Requested                  1
Funded                     1
Investor Funded            1
Number of Payments         1
Loan Grade             61270
Loan Subgrade          61270
Job                    23986
Years Employed         17538
Home                   61361
Annual Income          61028
Income Verified            1
Loan Date                  1
Reason                276440
Loan Cat                   1
Loan Title                19
State                      1
Ratio                      1
Late Payments              1
Credit Line Date           1
Months Del            218802
Months PR             348845
Derog Recs                 1
Credit Lines               1
Status                     1
dtype: int64

#### Drop rows with null values for Interest Rate (only drops about 15 percent).

In [53]:
loans.dropna(subset=['Interest Rate'], inplace=True)
loans.shape

(338990, 27)

#### Replace nulls in months since last deliquent and months since last public record with 0, assuming that null indicates 0 monhts.

In [54]:
loans.fillna(value={'Months PR': 0, 'Months Del': 0}, inplace=True)

In [55]:
loans['Months PR'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Months PR, dtype: float64

In [56]:
loans['Months Del'].head()

0     0.0
1     0.0
2    41.0
3    64.0
4    58.0
Name: Months Del, dtype: float64

#### Count unique values is columns and drop columns if large number of unique values (difficult to categorize)

Check Reason column

In [57]:
len(loans['Reason'].unique())

103870

In [58]:
loans=loans.drop('Reason', axis=1)

Check Job columns

In [59]:
len(loans['Job'].unique())

163396

In [60]:
loans=loans.drop('Job', axis=1)

In [61]:
loans.shape

(338990, 25)

Check Loan Title column

In [62]:
len(loans['Loan Title'].unique())

53615

In [63]:
loans=loans.drop('Loan Title', axis=1)

In [64]:
loans.shape

(338990, 24)

Check Id columns

In [65]:
len(loans['Borrower Id'].unique())

338990

In [66]:
loans.drop(['Loan Id', 'Borrower Id'], axis=1, inplace=True)

In [67]:
loans.shape

(338990, 22)

In [68]:
backup=loans.copy()

#### Analyze columns with string type

In [69]:
string_columns=loans.select_dtypes(include='object').columns
string_columns

Index(['Interest Rate', 'Requested', 'Funded', 'Investor Funded',
       'Number of Payments', 'Loan Grade', 'Loan Subgrade', 'Years Employed',
       'Home', 'Income Verified', 'Loan Date', 'Loan Cat', 'State',
       'Credit Line Date', 'Status'],
      dtype='object')

In [70]:
loans[string_columns].apply(lambda x: print(x.value_counts()))

10.99%    11082
12.99%     9951
15.61%     9869
8.90%      9656
7.90%      8569
          ...  
16.96%        1
24.40%        1
20.52%        1
14.67%        1
16.15%        1
Name: Interest Rate, Length: 482, dtype: int64
$10,000     24167
$12,000     18659
$15,000     16848
$20,000     16201
$35,000     11732
            ...  
$30,425         1
$33,850         1
$32,475         1
$33,050         1
$28,725         1
Name: Requested, Length: 1332, dtype: int64
$10,000     24087
$12,000     18582
$15,000     16752
$20,000     16047
$35,000     11617
            ...  
$33,850         1
$28,150         1
$26,725         1
$32,775         1
$22,775         1
Name: Funded, Length: 1333, dtype: int64
$10,000     20667
$12,000     15576
$15,000     13656
$20,000     12413
$8,000       9477
            ...  
$18,230         1
$34,995         1
$15,777         1
$404            1
$15,654         1
Name: Investor Funded, Length: 6403, dtype: int64
 36 months    247791
 60 months     91198
Name: 

Interest Rate         None
Requested             None
Funded                None
Investor Funded       None
Number of Payments    None
Loan Grade            None
Loan Subgrade         None
Years Employed        None
Home                  None
Income Verified       None
Loan Date             None
Loan Cat              None
State                 None
Credit Line Date      None
Status                None
dtype: object

Change all strings to upper case

In [71]:
loans[string_columns]=loans[string_columns].apply(lambda x: x.str.upper())

Change currency columns from string to float after removing right white space, commas, and $ symbol

In [72]:
dollar_columns=['Requested', 'Funded', 'Investor Funded']

In [73]:
loans[dollar_columns]=loans[dollar_columns].apply(lambda x: x.str.rstrip())
loans[dollar_columns]=loans[dollar_columns].apply(lambda x: x.str.replace(',', ''))
loans[dollar_columns]=loans[dollar_columns].apply(lambda x: x.str.replace('$', '').astype(float))

Change percent column to float

In [74]:
loans['Interest Rate']=loans['Interest Rate'].str.replace('%', '').astype(float)
loans['Interest Rate']=loans['Interest Rate']/100

#### Create method to convert date strings to date objects.

In [75]:
def fix_dates(string):
    fixed_date=string
    if isinstance(fixed_date, str) and len(fixed_date)>1:
        if fixed_date[0].isdigit():
            split=fixed_date.split('-')
            y=split[0]
            if len(y)==1:
                y='0'+y
            fixed_date=split[1]+'-'+y
        date_object=datetime.strptime(fixed_date, '%b-%y')
        if date_object.year>2040:
            date_object = date_object - relativedelta(years=100)
        return date_object

Change date strings to date objects

In [76]:
loans['Loan Date']=loans['Loan Date'].apply(lambda x: fix_dates(x))
loans['Credit Line Date']=loans['Credit Line Date'].apply(lambda x: fix_dates(x))

In [77]:
loans['Credit Line Date'].dt.year.value_counts(ascending=True)

1944.0        1
1946.0        1
1949.0        1
1951.0        1
1953.0        3
          ...  
1998.0    20715
2002.0    20960
1999.0    23976
2001.0    25013
2000.0    26222
Name: Credit Line Date, Length: 64, dtype: int64

In [78]:
loans['Loan Date'].dt.year.value_counts()

2014.0    145681
2013.0    114219
2012.0     45289
2011.0     18246
2010.0      9792
2009.0      4008
2008.0      1517
2007.0       237
Name: Loan Date, dtype: int64

In [79]:
loans.head()

,Interest Rate,Requested,Funded,Investor Funded,Number of Payments,Loan Grade,Loan Subgrade,Years Employed,Home,Annual Income,...,Loan Cat,State,Ratio,Late Payments,Credit Line Date,Months Del,Months PR,Derog Recs,Credit Lines,Status
0,0.1189,25000.0,25000.0,19080.0,36 MONTHS,B,B4,< 1 YEAR,RENT,85000.0,...,DEBT_CONSOLIDATION,CA,19.48,0.0,1994-02-01,0.0,0.0,0.0,42.0,F
1,0.1071,7000.0,7000.0,673.0,36 MONTHS,B,B5,< 1 YEAR,RENT,65000.0,...,CREDIT_CARD,NY,14.29,0.0,2000-10-01,0.0,0.0,0.0,7.0,F
2,0.1699,25000.0,25000.0,24725.0,36 MONTHS,D,D3,1 YEAR,RENT,70000.0,...,DEBT_CONSOLIDATION,NY,10.50,0.0,2000-06-01,41.0,0.0,0.0,17.0,F
3,0.1311,1200.0,1200.0,1200.0,36 MONTHS,C,C2,10+ YEARS,OWN,54000.0,...,DEBT_CONSOLIDATION,TX,5.47,0.0,1985-01-01,64.0,0.0,0.0,31.0,F
4,0.1357,10800.0,10800.0,10692.0,36 MONTHS,C,C3,6 YEARS,RENT,32000.0,...,DEBT_CONSOLIDATION,CT,11.63,0.0,1996-12-01,58.0,0.0,0.0,40.0,F


In [80]:
loans.dtypes

Interest Rate                float64
Requested                    float64
Funded                       float64
Investor Funded              float64
Number of Payments            object
Loan Grade                    object
Loan Subgrade                 object
Years Employed                object
Home                          object
Annual Income                float64
Income Verified               object
Loan Date             datetime64[ns]
Loan Cat                      object
State                         object
Ratio                        float64
Late Payments                float64
Credit Line Date      datetime64[ns]
Months Del                   float64
Months PR                    float64
Derog Recs                   float64
Credit Lines                 float64
Status                        object
dtype: object

#### Check categorical values of remainng categorical columns

In [81]:
loans['Loan Cat'].value_counts()

DEBT_CONSOLIDATION    198226
CREDIT_CARD            75680
HOME_IMPROVEMENT       19625
OTHER                  17154
MAJOR_PURCHASE          7312
SMALL_BUSINESS          5359
CAR                     4115
MEDICAL                 3329
MOVING                  2138
WEDDING                 1934
VACATION                1848
HOUSE                   1723
EDUCATIONAL              279
RENEWABLE_ENERGY         267
Name: Loan Cat, dtype: int64

In [82]:
loans.shape

(338990, 22)

In [83]:
loans['Home'].head()

0    RENT
1    RENT
2    RENT
3     OWN
4    RENT
Name: Home, dtype: object

In [84]:
loans['Home'].value_counts()

MORTGAGE    145958
RENT        115958
OWN          24976
OTHER          107
NONE            30
ANY              1
Name: Home, dtype: int64

#### Because the only valid values of Home, according to the metadata, are Mortgage, Rent, Own, and Other, the values of None and Any are changed to Other, which only affects 31 rows.

In [85]:
loans.loc[((loans['Home']=='ANY') | (loans['Home']=='NONE')),'Home']='OTHER'

In [86]:
loans['Home'].value_counts()

MORTGAGE    145958
RENT        115958
OWN          24976
OTHER          138
Name: Home, dtype: int64

In [87]:
loans.shape

(338990, 22)

In [88]:
loans['Status'].value_counts()

F    232600
W    106389
Name: Status, dtype: int64

In [89]:
loans['Years Employed'].value_counts()

10+ YEARS    108491
2 YEARS       30117
3 YEARS       26670
< 1 YEAR      26003
5 YEARS       23072
1 YEAR        21432
4 YEARS       20259
6 YEARS       19601
7 YEARS       19445
8 YEARS       16212
9 YEARS       12893
Name: Years Employed, dtype: int64

In [90]:
loans['Income Verified'].value_counts()

VERIFIED - INCOME           127040
NOT VERIFIED                107873
VERIFIED - INCOME SOURCE    104076
Name: Income Verified, dtype: int64

In [91]:
loans['State'].value_counts()

CA    52835
NY    29226
TX    26493
FL    22756
IL    13483
NJ    13188
PA    11877
OH    11040
GA    10851
VA    10338
NC     9303
MI     8350
MA     8038
MD     8016
AZ     7747
WA     7706
CO     7119
MN     5865
MO     5398
CT     5243
NV     4753
IN     4608
OR     4411
WI     4245
TN     4215
AL     4188
LA     4017
SC     3979
KY     3173
KS     3096
OK     3014
AR     2532
UT     2531
NM     1848
HI     1798
WV     1738
NH     1648
RI     1486
DC     1080
MT      995
AK      947
DE      896
WY      853
SD      730
MS      707
VT      604
ID        8
IA        7
NE        6
ME        4
Name: State, dtype: int64

In [92]:
missing_data = loans.isnull()
null_columns=loans.columns[missing_data.any()]
loans[null_columns].isnull().sum()

Requested                 1
Funded                    1
Investor Funded           1
Number of Payments        1
Loan Grade            51867
Loan Subgrade         51867
Years Employed        14795
Home                  51960
Annual Income         51752
Income Verified           1
Loan Date                 1
Loan Cat                  1
State                     1
Ratio                     1
Late Payments             1
Credit Line Date          1
Derog Recs                1
Credit Lines              1
Status                    1
dtype: int64

In [93]:
loans.shape

(338990, 22)

In [94]:
backup=loans.copy()

#### Because Loan Grade and Loan Subgrade have the same number of null values, check to see if these are in the same rows

In [95]:
len(loans[loans['Loan Grade'].isnull() & loans['Loan Subgrade'].isnull()])

51867

Since nulls are in both columns on the same rows, drop rows with nulls in both columns

In [96]:
loans=loans.dropna(subset=['Loan Grade', 'Loan Subgrade'])

In [97]:
loans.shape

(287123, 22)

Check for nulls after dropping rows

In [98]:
missing_data = loans.isnull()
null_columns=loans.columns[missing_data.any()]
loans[null_columns].isnull().sum()

Years Employed    12451
Home              44128
Annual Income     43990
dtype: int64

For Years Employed, fill nulls with the most common value

In [99]:
loans['Years Employed'].value_counts()/len(loans)

10+ YEARS    0.319967
2 YEARS      0.088986
3 YEARS      0.078757
< 1 YEAR     0.076723
5 YEARS      0.067964
1 YEAR       0.063151
4 YEARS      0.059974
6 YEARS      0.057731
7 YEARS      0.057282
8 YEARS      0.047969
9 YEARS      0.038130
Name: Years Employed, dtype: float64

In [100]:
loans.loc[:, 'Years Employed']=loans.loc[:, 'Years Employed'].fillna('10+ YEARS')

Because Home value doesn't have a dominant value, use forward fill to replace null values

In [101]:
loans['Home'].value_counts()/len(loans)

MORTGAGE    0.430112
RENT        0.342285
OWN         0.073488
OTHER       0.000425
Name: Home, dtype: float64

In [102]:
loans.loc[:, 'Home']=loans.loc[:, 'Home'].fillna(method='ffill')

Fill nulls in Annual Income with the mean annual income value

In [103]:
income_mean=loans['Annual Income'].mean()
loans.loc[:, 'Annual Income']=loans.loc[:, 'Annual Income'].fillna(value=income_mean)

In [104]:
loans.shape

(287123, 22)

In [105]:
loans.head()

,Interest Rate,Requested,Funded,Investor Funded,Number of Payments,Loan Grade,Loan Subgrade,Years Employed,Home,Annual Income,...,Loan Cat,State,Ratio,Late Payments,Credit Line Date,Months Del,Months PR,Derog Recs,Credit Lines,Status
0,0.1189,25000.0,25000.0,19080.0,36 MONTHS,B,B4,< 1 YEAR,RENT,85000.0,...,DEBT_CONSOLIDATION,CA,19.48,0.0,1994-02-01,0.0,0.0,0.0,42.0,F
1,0.1071,7000.0,7000.0,673.0,36 MONTHS,B,B5,< 1 YEAR,RENT,65000.0,...,CREDIT_CARD,NY,14.29,0.0,2000-10-01,0.0,0.0,0.0,7.0,F
2,0.1699,25000.0,25000.0,24725.0,36 MONTHS,D,D3,1 YEAR,RENT,70000.0,...,DEBT_CONSOLIDATION,NY,10.50,0.0,2000-06-01,41.0,0.0,0.0,17.0,F
3,0.1311,1200.0,1200.0,1200.0,36 MONTHS,C,C2,10+ YEARS,OWN,54000.0,...,DEBT_CONSOLIDATION,TX,5.47,0.0,1985-01-01,64.0,0.0,0.0,31.0,F
4,0.1357,10800.0,10800.0,10692.0,36 MONTHS,C,C3,6 YEARS,RENT,32000.0,...,DEBT_CONSOLIDATION,CT,11.63,0.0,1996-12-01,58.0,0.0,0.0,40.0,F


#### Save cleaned data to new csv file.

In [ ]:
loans.to_csv(r'data\clean_loan_interest_rates.csv', index=False)